In [2]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import os

# Load the CSV files
data_params = pd.read_csv('DataA_params.csv', sep=';')
hh_demand_true = pd.read_csv('hh_demand_true_A.csv')
weather_data = pd.read_csv('Munich_weather_1min.csv')

# Print the first few rows of each dataframe to ensure they are loaded correctly
print("Data Parameters:")
print(data_params.head())

print("\nHousehold Demand True:")
print(hh_demand_true.head())

print("\nWeather Data:")
print(weather_data.head())

# Set the NEOS email environment variable
os.environ['NEOS_EMAIL'] = 'jihad.jundi@tum.de'

# Extract parameter values from the data_params DataFrame
params = {row.iloc[0]: float(row.iloc[1]) for _, row in data_params.iterrows()}
params['c_M_PV'] = params.pop('c_MPV')  # Correct the parameter name

# Rename columns for clarity
weather_data.columns = ['Timestamp', 'G_Gk', 'Ta']
hh_demand_true.columns = ['Load']

# Generate timestamps for household demand data
hh_demand_true['Timestamp'] = pd.date_range(start='2013-01-10 00:00:00', periods=len(hh_demand_true), freq='min')

# Merge the data on timestamp
weather_data['Timestamp'] = pd.to_datetime(weather_data['Timestamp'])
merged_data = pd.merge(hh_demand_true, weather_data, on='Timestamp')

# Print the merged dataframe to ensure it is correct
print("\nMerged Data:")
print(merged_data.head())

# Define the parameters
alpha = params['alpha']
eta_ref = params['eta_ref']
beta = params['beta']
T_C_ref = params['T_cref']
gamma = params['gamma']
G_ref = params['G_ref']
h = params['h']
eta_B = params['eta_B']
c_B = params['c_B']
c_M_PV = params['c_M_PV']
c_PV = params['c_PV']
i_INV = params['i_INV']
p_EL = params['p_EL']
p_FI = params['p_FI']

# Pyomo model
model = pyo.ConcreteModel()

# Decision variables for PV capacity and battery capacity
model.CAP_PV = pyo.Var(bounds=(0, None), initialize=10)  # in kWp
model.CAP_B = pyo.Var(bounds=(0, None), initialize=15)  # in kWh

# Index for time periods
T = range(len(merged_data))
model.T = pyo.Set(initialize=T)

# Parameters from the merged data
P_HH = merged_data['Load'].values
G = merged_data['G_Gk'].values
T_a = merged_data['Ta'].values

# Variables for calculations
model.P_pv = pyo.Var(model.T, initialize=0)  # PV power output
model.P_B = pyo.Var(model.T, initialize=0)  # Battery power output
model.P_B_charge = pyo.Var(model.T, bounds=(0, None), initialize=0)  # Battery charging power
model.P_B_discharge = pyo.Var(model.T, bounds=(0, None), initialize=0)  # Battery discharging power
model.SOC = pyo.Var(model.T, bounds=(0, 1), initialize=0)  # State of charge of the battery
model.P_d = pyo.Var(model.T)  # Power difference (demand - supply)
model.E_D_plus_T = pyo.Var(model.T)  # Excess demand
model.E_D_minus_T = pyo.Var(model.T)  # Excess supply
model.T_c_t = pyo.Var(model.T)  # Cell temperature of the PV module
model.eta_pv_t = pyo.Var(model.T)  # Efficiency of the PV module

# Constraints for PV power
def PV_power_rule(model, t):
    # Calculate the area of the PV module
    A_c = alpha ** -1 * model.CAP_PV
    # Calculate the cell temperature
    model.T_c_t[t] = T_a[t] + h * G[t]
    # Calculate the efficiency of the PV module
    if G[t] > 0:
        model.eta_pv_t[t] = eta_ref * (1 - beta * (model.T_c_t[t] - T_C_ref) + gamma * pyo.log(G[t] / G_ref))
    else:
        model.eta_pv_t[t] = 0
    # Calculate the power output of the PV module
    return model.P_pv[t] == A_c * G[t] * model.eta_pv_t[t] * 1 / 1000

model.PV_power = pyo.Constraint(model.T, rule=PV_power_rule)

# Constraints for battery power (charging and discharging)
def Battery_power_rule(model, t):
    model.P_d[t] = P_HH[t] - model.P_pv[t]
    return model.P_B[t] == model.P_B_charge[t] - model.P_B_discharge[t]

model.Battery_power = pyo.Constraint(model.T, rule=Battery_power_rule)

def Battery_charge_rule(model, t):
    SOC_t_minus_1 = model.SOC[t-1] if t > 0 else 0
    max_charge = (1 - SOC_t_minus_1) * (model.CAP_B * (60 / h)) / eta_B
    return model.P_B_charge[t] <= max_charge

model.Battery_charge = pyo.Constraint(model.T, rule=Battery_charge_rule)

def Battery_discharge_rule(model, t):
    max_discharge = model.CAP_B * (60 / h)
    return model.P_B_discharge[t] <= max_discharge

model.Battery_discharge = pyo.Constraint(model.T, rule=Battery_discharge_rule)

# Constraints for state of charge (SOC)
def SOC_rule(model, t):
    if t == 0:
        return model.SOC[t] == 0
    else:
        return model.SOC[t] == model.SOC[t-1] + (1 / (eta_B * model.CAP_B * (60 / h))) * model.P_B_charge[t] - (1 / (model.CAP_B * (60 / h))) * model.P_B_discharge[t]

model.SOC_constr = pyo.Constraint(model.T, rule=SOC_rule)
# Define intermediate variables for E_D_plus_T and E_D_minus_T
model.E_D_plus_T_var = pyo.Var(model.T, initialize=0)
model.E_D_minus_T_var = pyo.Var(model.T, initialize=0)

# Constraints for excess demand (E_D_plus_T) and excess supply (E_D_minus_T)
def E_D_plus_T_rule(model, t):
    return model.E_D_plus_T_var[t] >= model.P_d[t]

def E_D_plus_T_zero_rule(model, t):
    return model.E_D_plus_T_var[t] >= 0

def E_D_minus_T_rule(model, t):
    return model.E_D_minus_T_var[t] >= -model.P_d[t]

def E_D_minus_T_zero_rule(model, t):
    return model.E_D_minus_T_var[t] >= 0

def E_D_plus_T_calc_rule(model, t):
    return model.E_D_plus_T[t] == model.E_D_plus_T_var[t] * (1/60)

def E_D_minus_T_calc_rule(model, t):
    return model.E_D_minus_T[t] == model.E_D_minus_T_var[t] * (1/60)

model.E_D_plus_T_constr1 = pyo.Constraint(model.T, rule=E_D_plus_T_rule)
model.E_D_plus_T_constr2 = pyo.Constraint(model.T, rule=E_D_plus_T_zero_rule)
model.E_D_minus_T_constr1 = pyo.Constraint(model.T, rule=E_D_minus_T_rule)
model.E_D_minus_T_constr2 = pyo.Constraint(model.T, rule=E_D_minus_T_zero_rule)
model.E_D_plus_T_calc_constr = pyo.Constraint(model.T, rule=E_D_plus_T_calc_rule)
model.E_D_minus_T_calc_constr = pyo.Constraint(model.T, rule=E_D_minus_T_calc_rule)

# Objective function to maximize the Net Present Value (NPV)
def NPV_rule(model):
    # Initial investment costs for PV and battery
    C_0_PV = c_PV * model.CAP_PV  # €
    C_0_B = c_B * model.CAP_B  # €
    # Annual maintenance cost for PV
    C_M_PV = c_M_PV * C_0_PV  # €
    
    NPV = -C_0_PV - C_0_B  # Initial costs (negative, as they are expenditures)
    
    # Sum up the total household demand over one year
    E_HH_T = sum(P_HH[t] * (1/60) for t in model.T)  # kWh
    
    # Calculate annual revenue over the 20-year period
    for T in range(1, 21):
        # Calculate the total excess solar energy fed into the grid over one year
        E_D_minus_T = sum(model.E_D_minus_T[t] for t in model.T)  # kWh
        # Calculate the total energy drawn from the grid over one year
        E_D_plus_T = sum(model.E_D_plus_T[t] for t in model.T)  # kWh
        
        # Calculate annual revenue
        R_T = E_D_minus_T * p_FI + (E_HH_T - E_D_plus_T) * p_EL  # €
        
        # Discount annual revenue and subtract annual maintenance cost
        NPV += (R_T - C_M_PV) / (1 + i_INV) ** T
    
    return NPV

model.NPV = pyo.Objective(rule=NPV_rule, sense=pyo.maximize)

model.NPV = pyo.Objective(rule=NPV_rule, sense=pyo.maximize)

# Solver
solver = pyo.SolverManagerFactory('neos')
results = solver.solve(model, opt="knitro", tee=True)

# Results extraction and printing

# Results
CAP_PV_opt = pyo.value(model.CAP_PV)
CAP_B_opt = pyo.value(model.CAP_B)
NPV_opt = pyo.value(model.NPV)

# Print the optimal values
print(f"Optimal PV Capacity: {CAP_PV_opt} kWp")
print(f"Optimal Battery Capacity: {CAP_B_opt} kWh")
print(f"Optimal NPV: {NPV_opt}")


Data Parameters:
  Parameter    Value
0     alpha   0.2500
1   eta_ref   0.2100
2      beta   0.0048
3    T_cref  25.0000
4     gamma   0.1200

Household Demand True:
       Load
0  0.133093
1  0.105168
2  0.120157
3  0.129763
4  0.123635

Weather Data:
            Unnamed: 0      G_Gk        Ta
0  2013-01-10 00:00:00  0.000000 -5.601643
1  2013-01-10 00:01:00  0.003981 -5.594408
2  2013-01-10 00:02:00  0.005333 -5.576817
3  2013-01-10 00:03:00  0.000000 -5.594365
4  2013-01-10 00:04:00  0.003592 -5.608405

Merged Data:
       Load           Timestamp      G_Gk        Ta
0  0.133093 2013-01-10 00:00:00  0.000000 -5.601643
1  0.105168 2013-01-10 00:01:00  0.003981 -5.594408
2  0.120157 2013-01-10 00:02:00  0.005333 -5.576817
3  0.129763 2013-01-10 00:03:00  0.000000 -5.594365
4  0.123635 2013-01-10 00:04:00  0.003592 -5.608405
'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually in